<a href="https://colab.research.google.com/github/eileenblum/Lyric-Generator/blob/main/Metal_data_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import abc
from nltk.corpus import stopwords
from textblob import Word

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
##########################  1  #################################################
############# Loading the data #################################################
################################################################################
#establish working directory:
os.chdir(r'/content/drive/MyDrive/metal_lyrics')
os.getcwd()
entries = os.scandir(r'/content/drive/MyDrive/metal_lyrics')
entries

###############  Gather the Metal Files  ################################
#Make sure to change this path to where YOU are storing lyrics in your computer    
Metal_name_and_text = {}
count = 0
# Walking a directory tree and printing the names of the directories and files
for dirpath, dirnames, files in os.walk(r'/content/drive/MyDrive/metal_lyrics'):
    #print(f'Found directory: {dirpath}')
    for file_name in files:
        count = count + 1
        os.listdir()
        if count%1000 == 0:
          print(count)
        #print("song:", file_name)
        with open(dirpath + r'/' + file_name, "r", errors='ignore') as target_file:
            Metal_name_and_text[file_name] = target_file.read()

Metal_data = (pd.DataFrame.from_dict(Metal_name_and_text, orient='index')).reset_index().rename(index=str, columns={'index': 'song_name', 0: 'lyrics'})



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000


In [ ]:
# Getting rid of two non-song files at the beginning (lml_archive.txt and .DS_Store,
# and metal_lyrics.iml, modules.xml, workspace.xml, and profile_settings.xml at the end):

Metal_data = Metal_data[2:-4]
Metal_data.reset_index(inplace=True, drop=True)
Metal_data.head()

,song_name,lyrics
0,1. The Moulting (Intro).txt,"""Throughout the fluid mass, but downward purge..."
1,3. Feed Them Hate.txt,What lies above nourishes the below. The waste...
2,2. Vehement Indulgences.txt,Sickening lives. Throes of squalor. Hollowed l...
3,4. The Bottomless Perdition.txt,Womb of nature.\nSpoil and ruin.\nDeath yawn s...
4,3. Valley Of Defilement.txt,Rotten throne.\nCorrupted master.\nSending the...


In [ ]:
# Adding "type" and getting rid of instrumentals.

Type = np.ones(shape=(len(Metal_data),1 ))
Metal_data['type'] = Type

for i in Metal_data.index:
  if (Metal_data.loc[i,"lyrics"] == "instrumental") or (Metal_data.loc[i,"lyrics"] == ""):
    Metal_data.drop(i,inplace=True)

Metal_data.reset_index(inplace=True, drop=True)

In [ ]:
Metal_data.head()

,song_name,lyrics,type
0,1. The Moulting (Intro).txt,"""Throughout the fluid mass, but downward purge...",1.0
1,3. Feed Them Hate.txt,What lies above nourishes the below. The waste...,1.0
2,2. Vehement Indulgences.txt,Sickening lives. Throes of squalor. Hollowed l...,1.0
3,4. The Bottomless Perdition.txt,Womb of nature.\nSpoil and ruin.\nDeath yawn s...,1.0
4,3. Valley Of Defilement.txt,Rotten throne.\nCorrupted master.\nSending the...,1.0


In [ ]:
def cleaner_keep_newlines(df):
  df_copy = df.copy()
  df_copy['cleaned_lyrics'] = df_copy['lyrics'].apply(lambda x: ' '.join(x for x in x.split(" ") if x not in string.punctuation))#All the rows of the text in the data frame is checked for string punctuations, and these are filtered
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].str.replace(r'[^\w\s\n\r\t]','')#REmoves dots using regular expressions
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].apply(lambda x:' '.join(x.lower() for x in x.split(" "))) #Converting text to lower case
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].apply(lambda x: ' '.join(x for x in x.split(" ") if  not x.isdigit()))#Digits are removed from the text
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].apply(lambda x: ' '.join(x for x in x.split(" ") if not x in stop))#Stop words are removed at this stage
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split(" ")]))#Words are filtered now, and different form of the same word is removed using lemmatization

  return df_copy


In [ ]:
def cleaner(df):
  df_copy = df.copy()
  df_copy['cleaned_lyrics'] = df_copy['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))#All the rows of the text in the data frame is checked for string punctuations, and these are filtered
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].str.replace(r'[^\w\s]','')#REmoves dots using regular expressions
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].apply(lambda x:' '.join(x.lower() for x in x.split())) #Converting text to lower case
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))#Digits are removed from the text
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].apply(lambda x: ' '.join(x for x in x.split() if not x in stop))#Stop words are removed at this stage
  df_copy['cleaned_lyrics'] = df_copy['cleaned_lyrics'].apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))#Words are filtered now, and different form of the same word is removed using lemmatization

  return df_copy

In [ ]:
##########################  2  #################################################
############# Preprocessing the Data ###########################################
################################################################################
import re

Lyrics = Metal_data.copy()

Lyrics = Lyrics.dropna()

################# Cleaning ##############################
stop = stopwords.words('english') #importing English stop words
#cleaning and lemmatizing Lyrics:
Pro_Lyrics = cleaner_keep_newlines(Lyrics)
Pro_Lyrics_no_newlines = cleaner(Lyrics)

# Here we compile a list of words, with no newline characters
# (This no-newline-df won't be used again, as we need the newline
# characters for splitting-by-line in the data processing.)
Pro_Lyrics_list = []
for i in Pro_Lyrics_no_newlines['cleaned_lyrics']:
     li = list(i.split(" "))
     Pro_Lyrics_list.append(li)	
     
Pro_Lyrics['words'] = Pro_Lyrics_list #Adding the list of words/lemmas used for every song     

################################################################################
#######  END OF PREPROCESSING  #################################################
################################################################################

In [ ]:
for i in Pro_Lyrics.index:
  if (Pro_Lyrics.loc[i,"cleaned_lyrics"].replace(r'^[\w]','') == "instrumental") or (Pro_Lyrics.loc[i,"cleaned_lyrics"].replace(r'^[\w]','') == ""):
    Pro_Lyrics.drop(i,inplace=True)
  if (Pro_Lyrics.loc[i,"cleaned_lyrics"].replace('\n','') == "instrumental") or (Pro_Lyrics.loc[i,"cleaned_lyrics"].replace('\n','') == ""):
    Pro_Lyrics.drop(i,inplace=True)
    
Pro_Lyrics.reset_index(inplace=True, drop=True)

In [ ]:
Pro_Lyrics.sample(15)

,song_name,lyrics,type,cleaned_lyrics,words
8920,8. Wasted Life.txt,I wish I could dream without fear and\nNot to ...,1.0,wish could dream without fear and\nnot live ru...,"[wish, could, dream, without, fear, live, rule..."
43317,4. Haz Tu Jugada.txt,[Walter Giardino]\n\nDebes mostrar tu realidad...,1.0,walter giardino\n\ndebes mostrar tu realidad\n...,"[walter, giardino, debes, mostrar, tu, realida..."
24300,8. Faceless Queen.txt,"""The face of Tiamat, like the ocean's surface,...",1.0,face tiamat like ocean surface reflection ever...,"[face, tiamat, like, ocean, surface, reflectio..."
23302,7. Credence Of Fort William.txt,[Severe Dementia - Epitaph Of Plassey]\n\nNawa...,1.0,severe dementia epitaph plassey\n\nnawab thron...,"[severe, dementia, epitaph, plassey, nawab, th..."
16808,6. Warlike Conspiracy.txt,My vacant art just imitate\nA vacant nameless ...,1.0,vacant art imitate\na vacant nameless life\nth...,"[vacant, art, imitate, vacant, nameless, life,..."
55667,4. Reign Of The Ravenous.txt,"This is the sign for those\nWaiting, all seein...",1.0,sign those\nwaiting seeing\nwith wing spread w...,"[sign, waiting, seeing, wing, spread, wide, as..."
10299,8. You Run.txt,Heavy thoughts seem to slip away\nWhen you wer...,1.0,heavy thought seem slip away\nwhen darkest day...,"[heavy, thought, seem, slip, away, darkest, da..."
9948,8. Tomes Of Acrimony.txt,"Etched within the centuries, \nbequeathed upon...",1.0,etched within century \nbequeathed upon genera...,"[etched, within, century, bequeathed, upon, ge..."
46918,2. Leatherhead.txt,"[Kaufmann, Dirkschneider]\n\nJust Give Me A Re...",1.0,kaufmann dirkschneider\n\njust give reason\nan...,"[kaufmann, dirkschneider, give, reason, iâll, ..."
6891,10. Fag Basher.txt,Don't look back at your wasted like us don't e...,1.0,dont look back wasted like u dont even try vis...,"[dont, look, back, wasted, like, u, dont, even..."


In [ ]:
Pro_Lyrics_corpus = []

for L in Pro_Lyrics_list:
  for word in L:
    if word not in Pro_Lyrics_corpus:
      Pro_Lyrics_corpus.append(word)

In [9]:
# Corpus english word cleaning:
from nltk.corpus import words
nltk.download('words')

# This is our english dictionary:
english_dictionary_list = words.words()
print(*english_dictionary_list, sep = "\n")

Streaming output truncated to the last 5000 lines.
wherewith
wherewithal
wherret
wherrit
wherry
wherryman
whet
whether
whetile
whetrock
whetstone
whetter
whew
whewellite
whewer
whewl
whewt
whey
wheybeard
wheyey
wheyeyness
wheyface
wheyfaced
wheyish
wheyishness
wheylike
wheyness
whiba
which
whichever
whichsoever
whichway
whichways
whick
whicken
whicker
whid
whidah
whidder
whiff
whiffenpoof
whiffer
whiffet
whiffle
whiffler
whifflery
whiffletree
whiffling
whifflingly
whiffy
whift
Whig
whig
Whiggamore
whiggamore
Whiggarchy
Whiggery
Whiggess
Whiggification
Whiggify
Whiggish
Whiggishly
Whiggishness
Whiggism
Whiglet
Whigling
whigmaleerie
whigship
whikerby
while
whileen
whilere
whiles
whilie
whilk
Whilkut
whill
whillaballoo
whillaloo
whillilew
whilly
whillywha
whilock
whilom
whils
whilst
whilter
whim
whimberry
whimble
whimbrel
whimling
whimmy
whimper
whimperer
whimpering
whimperingly
whimsey
whimsic
whimsical
whimsicality
whimsically
whimsicalness
whimsied
whimstone
whimwham
whin
whinberry
whi

In [ ]:
Pro_Lyrics_corpus2 = []

for word in Pro_Lyrics_corpus:
  if word in english_dictionary_list:
    Pro_Lyrics_corpus2.append(word)

In [ ]:
import pickle

# saving corpus:
with open("/content/drive/MyDrive/NLP bootcamp project/Pro_Lyrics_corpus.txt", "wb") as fp:
  pickle.dump(Pro_Lyrics_corpus, fp)

# saving english-only corpus:
with open("/content/drive/MyDrive/NLP bootcamp project/Pro_Lyrics_corpus_english.txt", "wb") as fp:
  pickle.dump(Pro_Lyrics_corpus2, fp)

# saving word list (contains repeats):
with open("/content/drive/MyDrive/NLP bootcamp project/Pro Lyrics list.txt", "wb") as fp:
  pickle.dump(Pro_Lyrics_list, fp)

# saving dataframe:
Pro_Lyrics.to_csv("/content/drive/MyDrive/NLP bootcamp project/Pro_Lyrics_df.csv")
